<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Machine Learning Foundation

## Course 5, Part g: RNN DEMO


## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).


In [1]:
%%capture
# !pip install --upgrade tensorflow

In [2]:
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [3]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [4]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [5]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [6]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [7]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [8]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [9]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [10]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 73s 85ms/step - loss: 0.6552 - accuracy: 0.6330 - val_loss: 0.5976 - val_accuracy: 0.6908
Epoch 2/10
782/782 [==============================] - 63s 81ms/step - loss: 0.5512 - accuracy: 0.7232 - val_loss: 0.5382 - val_accuracy: 0.7275
Epoch 3/10
782/782 [==============================] - 64s 82ms/step - loss: 0.4912 - accuracy: 0.7639 - val_loss: 0.5089 - val_accuracy: 0.7463
Epoch 4/10
782/782 [==============================] - 65s 83ms/step - loss: 0.4528 - accuracy: 0.7903 - val_loss: 0.4924 - val_accuracy: 0.7570
Epoch 5/10
782/782 [==============================] - 65s 83ms/step - loss: 0.4247 - accuracy: 0.8067 - val_loss: 0.4728 - val_accuracy: 0.7702
Epoch 6/10
782/782 [==============================] - 63s 81ms/step - loss: 0.4044 - accuracy: 0.8162 - val_loss: 0.4664 - val_accuracy: 0.7742
Epoch 7/10
782/782 [==============================] - 63s 80ms/step - loss: 0.3889 - accuracy: 0.8254 - val_loss: 0.4566 - val_accuracy:

In [11]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 10s 13ms/step - loss: 0.4498 - accuracy: 0.7878
Test score: 0.4497670829296112
Test accuracy: 0.7877600193023682


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?


In [12]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [13]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [14]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 156s 192ms/step - loss: 0.6023 - accuracy: 0.6750 - val_loss: 0.5289 - val_accuracy: 0.7369
Epoch 2/10
782/782 [==============================] - 151s 193ms/step - loss: 0.4531 - accuracy: 0.7900 - val_loss: 0.4959 - val_accuracy: 0.7584
Epoch 3/10
782/782 [==============================] - 151s 193ms/step - loss: 0.3906 - accuracy: 0.8253 - val_loss: 0.4297 - val_accuracy: 0.8002
Epoch 4/10
782/782 [==============================] - 150s 192ms/step - loss: 0.3486 - accuracy: 0.8475 - val_loss: 0.3993 - val_accuracy: 0.8181
Epoch 5/10
782/782 [==============================] - 149s 190ms/step - loss: 0.3192 - accuracy: 0.8646 - val_loss: 0.3927 - val_accuracy: 0.8233
Epoch 6/10
782/782 [==============================] - 149s 190ms/step - loss: 0.2985 - accuracy: 0.8752 - val_loss: 0.3766 - val_accuracy: 0.8327
Epoch 7/10
782/782 [==============================] - 149s 191ms/step - loss: 0.2831 - accuracy: 0.8836 - val_loss: 0.3772 -

In [16]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [17]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [18]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [19]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 157s 194ms/step - loss: 0.6727 - accuracy: 0.5951 - val_loss: 0.6130 - val_accuracy: 0.6896
Epoch 2/10
782/782 [==============================] - 158s 202ms/step - loss: 0.5461 - accuracy: 0.7317 - val_loss: 0.5230 - val_accuracy: 0.7420
Epoch 3/10
782/782 [==============================] - 161s 206ms/step - loss: 0.4788 - accuracy: 0.7729 - val_loss: 0.4887 - val_accuracy: 0.7679
Epoch 4/10
782/782 [==============================] - 161s 206ms/step - loss: 0.4422 - accuracy: 0.7972 - val_loss: 0.4675 - val_accuracy: 0.7731
Epoch 5/10
782/782 [==============================] - 162s 208ms/step - loss: 0.4156 - accuracy: 0.8134 - val_loss: 0.4495 - val_accuracy: 0.7874
Epoch 6/10
782/782 [==============================] - 160s 205ms/step - loss: 0.3934 - accuracy: 0.8253 - val_loss: 0.4350 - val_accuracy: 0.7954
Epoch 7/10
782/782 [==============================] - 162s 207ms/step - loss: 0.3760 - accuracy: 0.8337 - val_loss: 0.4264 -

In [20]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 157s 200ms/step - loss: 0.3290 - accuracy: 0.8586 - val_loss: 0.3834 - val_accuracy: 0.8269
Epoch 2/10
782/782 [==============================] - 159s 203ms/step - loss: 0.3207 - accuracy: 0.8632 - val_loss: 0.3899 - val_accuracy: 0.8233
Epoch 3/10
782/782 [==============================] - 159s 204ms/step - loss: 0.3135 - accuracy: 0.8673 - val_loss: 0.3748 - val_accuracy: 0.8319
Epoch 4/10
782/782 [==============================] - 160s 205ms/step - loss: 0.3073 - accuracy: 0.8707 - val_loss: 0.3870 - val_accuracy: 0.8268
Epoch 5/10
782/782 [==============================] - 160s 205ms/step - loss: 0.3024 - accuracy: 0.8723 - val_loss: 0.3664 - val_accuracy: 0.8366
Epoch 6/10
782/782 [==============================] - 156s 199ms/step - loss: 0.2979 - accuracy: 0.8756 - val_loss: 0.3652 - val_accuracy: 0.8367
Epoch 7/10
782/782 [==============================] - 156s 200ms/step - loss: 0.2941 - accuracy: 0.8772 - val_loss: 0.3631 -

---
### Machine Learning Foundation (C) 2020 IBM Corporation
